In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [2]:
import csv
import os
import numpy as np

In [14]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp.solutions.pose # Mediapipe Solutions
mp_holistic = mp.solutions.holistic

In [12]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results.pose_landmarks.landmark)
        
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
                        
        cv2.imshow('Raw Webcam Feed', cv2.flip(image, 1))

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [6]:
num_coords = len(results.pose_landmarks.landmark)

In [7]:
# 좌표 저장을 위한 리스트 생성. 첫번째 항목은 분류할 동작의 클래스 이름.
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [8]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [22]:
class_name = "face2"

In [23]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_world_landmarks.landmark
            pose_row = list(np.array([["{:.7f}".format(landmark.x), "{:.7f}".format(landmark.y), "{:.7f}".format(landmark.z), "{:.7f}".format(landmark.visibility)] for landmark in pose]).flatten())
            
            # Concate rows
            row = pose_row
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', cv2.flip(image, 1))

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [21]:
np.array([["{:.7f}".format(landmark.x), "{:.7f}".format(landmark.y), "{:.7f}".format(landmark.z), "{:.7f}".format(landmark.visibility)] for landmark in results.pose_world_landmarks.landmark])

array([['0.0187321', '-0.6244495', '-0.2151067', '0.9961768'],
       ['0.0285928', '-0.6588701', '-0.1993116', '0.9966590'],
       ['0.0291491', '-0.6587456', '-0.1988231', '0.9966612'],
       ['0.0288567', '-0.6593617', '-0.1997995', '0.9967943'],
       ['-0.0029750', '-0.6568913', '-0.1970164', '0.9966219'],
       ['-0.0021894', '-0.6575819', '-0.1986185', '0.9965130'],
       ['-0.0031000', '-0.6584114', '-0.1989971', '0.9967361'],
       ['0.0741237', '-0.6346843', '-0.0993776', '0.9974866'],
       ['-0.0651290', '-0.6328152', '-0.1023725', '0.9966078'],
       ['0.0368719', '-0.5967139', '-0.1824631', '0.9934920'],
       ['-0.0031752', '-0.5938415', '-0.1810138', '0.9934879'],
       ['0.1608139', '-0.4448940', '-0.0463908', '0.9011457'],
       ['-0.1630973', '-0.4561642', '-0.0731364', '0.8985484'],
       ['0.1709440', '-0.3427214', '-0.1720033', '0.6621099'],
       ['-0.1761722', '-0.3523906', '-0.1868181', '0.6960315'],
       ['0.0690379', '-0.3954115', '-0.2797360',